In [1]:
from  projectCode.imports import *

In [2]:
device = set_device()
ann_file = "./annotations/annotations/captions_val2017.json"
num_examples = 512
batch_size = 1
seed = 42


cuda


In [3]:
ds = CocoDataset(ann_file, num_examples, img_dir = "imgs",seed = seed)
dl = DataLoader(ds, batch_size = batch_size)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Selected 512 images.


In [4]:
model_params={"device":device,"tokens_path":'./models/ExpansionNet_v2/demo_material/demo_coco_tokens.pickle',"blur_kernel_size":5,"blured":True}
model = loadModel("ExpansionNet",model_path="./models/ExpansionNet_v2/rf_model.pth",**model_params)

/home/captioning/miniconda3/envs/captioning/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1720538459595/work/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/captioning/236874/projectCode/ModelLoader.py:177: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loade

In [5]:
iter_per_attack = 25
base = "./adversarial_attacks"+"/ExpNet"+"/blured"+f"/it{iter_per_attack}_exNum{num_examples}"
attacks_file_name = base + ".pt" 
captions_file = base+".json"

In [6]:
# GENERATING ADVERSARIAL EXAMPLES
if not os.path.exists(attacks_file_name):
    attacker = Attacker(model,torch.nn.MSELoss(),device, num_iterations=iter_per_attack)
    attacks = []

    for batch in tqdm(dl):
        pert = attacker.perturbExpNet(batch[0])
        attacks.append(pert)
    perts = torch.cat(attacks)
    torch.save(perts,attacks_file_name)
else:
    perts = torch.load(attacks_file_name)


  0%|          | 1/512 [00:22<3:09:16, 22.22s/it]


KeyboardInterrupt: 

In [ ]:
captions = None
caps = None

#use if exists already
if  os.path.exists(captions_file):
    with open(captions_file, 'r') as openfile:
        captions = json.load(openfile)
        if "captions" in captions.keys():
            caps = captions["captions"]
    
if caps is None:
    cas = []
    for i in range(num_examples):
        x = ds[i]
        pert = perts[i]
        
        out, _, _, _ = model(x[0].to(device)) # prediction, pred_probs, logits, cross_enc_output
        print(out)
        p_out, _, _, _ = model(torch.clamp(x[0].to(device)+pert.to(device),0,1))
        caps.append((out,p_out))

    if captions is None:
        captions =  {
        "seed": seed,
        "num_examples": num_examples,
        "batch_size": batch_size,
        "captions": caps
        }
    else:
        captions["captions"] = caps

    with open(captions_file, 'w') as openfile:
            json.dump(captions,openfile)

/tmp/ipykernel_304160/1313045100.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  perts = torch.load(f"ExpNet_{num_examples}_pert_v2.pt")


In [ ]:
#use if already exists
if "scores" in captions.keys():
    meteor_scores = captions["scores"]["meteor"]

else:
    meteor_scores=[]
    for caps,ex in zip(caps,ds):
        c_clear = caps[0]
        c_corr = caps[1]
        img = ex[0]

        anot = [an[0] for an in ex[1]]
        
        score =  meteor_eval_single(anot,c_clear[0])
        score_corr = meteor_eval_single(anot,c_corr[0])
        
        
        meteor_scores.append((score,score_corr))#,out,p_out)) #img, delta, model_output, model_output_corrupted

    captions["scores"]={}
    captions["scores"]["meteor"] = meteor_scores

    with open(captions_file, 'w') as openfile:
        json.dump(captions,openfile)

FileNotFoundError: [Errno 2] No such file or directory: './ExpNet_512_1_cap_v2.json'

In [ ]:
print("attack statistics")
t = torch.tensor([list(p) for p in meteor_scores])
diff = t[:,0]-t[:,1]
avg = diff.mean()
print("Average reduction in meteor score:",avg)
improve = (diff<0)
print("Number of samples that improved score:",improve.sum(), "out of 512")
print("The average improvement rate:",-diff[improve].mean())
filter = t[:,0]>0.45
tf = t[filter]
diff = tf[:,0]-tf[:,1]
avg = diff.mean()
print("Average reduction in meteor score for images with high initial score:",avg)
improve = (diff<0)
print("Number of samples that improved high initial score:",improve.sum(), "out of 512")
print("The average improvement rate:",-diff[improve].mean())